# 1. Get S&P data from Yahoo Finance

In [15]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import os
import pandas_ta
import warnings

warnings.filterwarnings('ignore')

# Define a file path to save and load the data
data_file = 'stock_data.csv'

# Check if the data file exists, and load it if it does
if os.path.exists(data_file):
  df = pd.read_csv(data_file, index_col=[0, 1], header=0, parse_dates=True)
else:
  # If the data file doesn't exist, download the data
  sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

  # symbols cleanup - some symbols contain dots ".", we need to replace them with "-" so we can read data with yfinance
  sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

  # grab list with all the stock symbols
  symbols_list = sp500['Symbol'].unique().tolist()

  # Remove 'VLTO' from the list - it does not have enough data for later calculations
  symbols_list.remove('VLTO')

  end_date = '2023-10-31'
  start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8) # 8 years ago

  # download data from yahooFinance
  df = yf.download(tickers=symbols_list,
    start=start_date,
    end=end_date).stack()


  # Save the downloaded data to a file
  df.to_csv(data_file)


# add/change column names
df.index.names = ['date', 'ticker']
# fix column names to lower case
df.columns = df.columns.str.lower()

df

[*********************100%%**********************]  502 of 502 completed


5 Failed downloads:
['VTR', 'EXPD', 'SEE', 'WRB', 'SWKS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2015-11-02 00:00:00 -> 2023-10-31)')


adj close       close        high         low        open  \
date       ticker                                                               
2015-11-02 A        36.128700   38.590000   38.619999   37.799999   37.869999   
           AAL      44.313286   46.470001   46.820000   46.200001   46.200001   
           AAPL     27.498981   30.295000   30.340000   29.902500   30.200001   
           ABBV     44.862259   63.380001   64.199997   61.439999   61.599998   
           ABT      39.108421   45.430000   45.500000   44.599998   44.880001   
...                       ...         ...         ...         ...         ...   
2023-10-30 YUM     119.870003  119.870003  120.639999  119.260002  120.290001   
           ZBH     103.410004  103.410004  104.099998  102.330002  103.760002   
           ZBRA    209.770004  209.770004  211.210007  204.470001  207.500000   
           ZION     29.980000   29.980000   30.180000   29.320000   29.840000   
           ZTS     156.625000  157.000000  158.410004  153.520004  157.130005   

                        volume  
date       ticker               
2015-11-02 A         1810800.0  
           AAL       6189300.0  
           AAPL    128813200.0  
           ABBV     17008700.0  
           ABT       5477800.0  
...                        ...  
2023-10-30 YUM       1551900.0  
           ZBH       1309800.0  
           ZBRA       970000.0  
           ZION      2837100.0  
           ZTS       2756200.0  

[984474 rows x 6 columns]

# 2. Calculate features and technical indicators for each stock
- Garman-Klass Volatility (intraday volatility for a given asset)
- RSI
- Bollinger Bands
- ATR
- MACD
- Dollar Volume

Garman-Klass Volatility
\begin{equation}
\frac{(ln(High) - ln(Low))^{2}}{2}-(2ln(2)-1)(ln(\text{Adj Close})-ln(Open))^{2}
\end{equation}



In [17]:
## Garman-Klass Volatility
df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low']))**2)/2-(2*np.log(2)-1)*(np.log(df['adj close'])-np.log(df['open']))**2

## RSI - groupby (levl=1) == ticker
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

## Bollinger Bands - low - mid - high
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

df

adj close       close        high         low        open  \
date       ticker                                                               
2015-11-02 A        36.128700   38.590000   38.619999   37.799999   37.869999   
           AAL      44.313286   46.470001   46.820000   46.200001   46.200001   
           AAPL     27.498981   30.295000   30.340000   29.902500   30.200001   
           ABBV     44.862259   63.380001   64.199997   61.439999   61.599998   
           ABT      39.108421   45.430000   45.500000   44.599998   44.880001   
...                       ...         ...         ...         ...         ...   
2023-10-30 YUM     119.870003  119.870003  120.639999  119.260002  120.290001   
           ZBH     103.410004  103.410004  104.099998  102.330002  103.760002   
           ZBRA    209.770004  209.770004  211.210007  204.470001  207.500000   
           ZION     29.980000   29.980000   30.180000   29.320000   29.840000   
           ZTS     156.625000  157.000000  158.410004  153.520004  157.130005   

                        volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-11-02 A         1810800.0         -0.000626        NaN       NaN   
           AAL       6189300.0         -0.000583        NaN       NaN   
           AAPL    128813200.0         -0.003286        NaN       NaN   
           ABBV     17008700.0         -0.037869        NaN       NaN   
           ABT       5477800.0         -0.007120        NaN       NaN   
...                        ...               ...        ...       ...   
2023-10-30 YUM       1551900.0          0.000061  43.110984  4.765199   
           ZBH       1309800.0          0.000143  33.314329  4.620582   
           ZBRA       970000.0          0.000480  38.278362  5.283422   
           ZION      2837100.0          0.000409  39.613464  3.376202   
           ZTS       2756200.0          0.000488  27.979275  5.059724   

                     bb_mid   bb_high  
date       ticker                      
2015-11-02 A            NaN       NaN  
           AAL          NaN       NaN  
           AAPL         NaN       NaN  
           ABBV         NaN       NaN  
           ABT          NaN       NaN  
...                     ...       ...  
2023-10-30 YUM     4.793638  4.822076  
           ZBH     4.675267  4.729951  
           ZBRA    5.366760  5.450097  
           ZION    3.520557  3.664912  
           ZTS     5.134859  5.209994  

[984474 rows x 11 columns]